In [1]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from transformers import BertTokenizer, BertModel
import json
import re
import logging
from transformers import BertModel, BertTokenizer

from torch.utils.data import DataLoader, Dataset
from sklearn.cluster import DBSCAN, KMeans
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from pymystem3.mystem import Mystem
import re
import nltk
import numpy as np
import pandas as pd=

nltk.download('stopwords')
from nltk.corpus import stopwords

ModuleNotFoundError: No module named 'pymystem3'

In [ ]:
pronouns = "я, мы, ты, вы, он, она, оно, они, себя, мой, твой, ваш, наш, свой, его, ее, их, то, это, " \
                        "тот, этот, такой, таков, столько, весь, всякий, сам, самый, каждый, любой, иной, другой, " \
                        "кто, что, какой, каков, чей, сколько, никто, ничто, некого, нечего, никакой, ничей, " \
                        "нисколько, кто-то, кое-кто, кто-нибудь, кто-либо, что-то, кое-что, что-нибудь, что-либо, " \
                        "какой-то, какой-либо, какой-нибудь, некто, нечто, некоторый, некий, который".split(", ")
particles = "в, без, до, для, за, через, над, по, из, у, около, под, о, про, на, к, перед, при, с, " \
                  "между, а, абы, аж, ан, благо, буде, будто, вроде, да, дабы, даже, едва, ежели, если, же, " \
                  "затем, зато, и, ибо, или, итак, кабы, как, когда, коли, коль, ли, либо, лишь, нежели, но, " \
                  "пока, покамест, покуда, поскольку, притом, причем, пускай, пусть, раз, разве, ровно, " \
                  "сиречь, словно, так, также, тоже, только, точно, хоть, хотя, чем, чисто, что, чтоб, чтобы, " \
                  "чуть, якобы, да, ни, тоже, также, не, только, но, и, столько, сколько, то, чтобы, не, то, " \
                  "чтобы, а, зато, однако, или, либо, когда, едва, пока, как, только, лишь, потому, что, так, " \
                  "как, ибо, оттого, чтобы, для, того, с, тем, при, ещё, ой".split(", ")
stopwords = np.unique(np.hstack((stopwords.words("russian"), pronouns, particles)))

In [ ]:
import os
dfs = [pd.read_csv(os.path.join('../input/vkgroupsvtb', f)) for f in os.listdir('../input/vkgroupsvtb')[1:]]

In [ ]:
# dfs[3].replace('Консультант Плюс - законодательство РФ кодексы и законы в последней редакции. Удобный поиск законов кодексов приказов и других документов. Ежедневные обзоры законов. Консультации по бухучету и налогообложению.', 
#                np.NaN, inplace=True)

In [ ]:
df = pd.concat(dfs, ignore_index=True)

In [ ]:
# df.to_csv('data.csv')

In [ ]:
os.remove('data.csv')

In [ ]:
# df = pd.read_csv('../input/vkgroupsvtb/incrussiamedia.csv')

In [ ]:
df

In [ ]:
text_feature = 'link_description'

In [ ]:
df[text_feature].str.len().describe()

In [ ]:
!pip install pymystem3

In [ ]:
from pymystem3.mystem import Mystem

lemmatizer = Mystem()

In [ ]:
def preproc(text):
    text = text.lower()
    text = re.sub(r"http(?:s)?:[\/\\]{2}\w+(?:.[\w-]+)+(?:[\/\\]{1}[\w%\(\)_\\\.\-~]*)*", " ", text)
    text = re.sub(r"[\W_\d ]+", " ", text)
    text = re.sub(r"\s+", " ", text)
    text = " ".join(list(filter(lambda x: len(x) > 2 and x not in stopwords, 
                                lemmatizer.lemmatize(text))))
    return text

In [ ]:
df = df.dropna(subset=['text', 'link_description']).drop(columns=['timereposts', 'Unnamed: 0'])

In [ ]:
df[['preproc_text', 'preproc_description']] = df[['text', 'link_description']].applymap(preproc)

In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, tokenizer, data, token_len: int):
        self.texts = data
        self.tokenizer = tokenizer
        self.token_len = token_len

    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return self.tokenizer(self.texts[idx], padding='max_length', 
                              max_length=self.token_len, truncation=True, return_tensors="pt")

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
df[text_feature].str.len().describe()

In [ ]:
df = df.drop_duplicates(subset=['preproc_text'])

In [ ]:
df = df.drop_duplicates(subset=['preproc_description'])

In [ ]:
df

In [ ]:
# def get_embeddings(model_checkpoint, texts, token_len=256, batch_size=256):
#     """
#     Get embeddings for texts

#     Parameters
#     ----------
#         model_checkpoint: str
#         List of maximum distances between two samples of cluster points to try
#     min_samples_grid : list of int, default=[100]
#         List of neighbor sample count for a point to be considered core
#     metric : str, default='cosine'
#         The metric to use when calculating distance between instances in a feature array
#     column : str, default='PreprocessedText'
#         Name of DataFrame column. Contains texts, which are to be clusterized
#     """
#     tokenizer = BertTokenizer.from_pretrained(model_checkpoint)
#     model = BertModel.from_pretrained(model_checkpoint).to(device)
#     dataset = Dataset(tokenizer, texts, token_len)
#     dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
#     model.eval()
#     with torch.no_grad():
#         for i, batch in enumerate(tqdm(dataloader)):
#             masks = batch['attention_mask'].to(device)
#             input_ids = batch['input_ids'].squeeze(1).to(device)
#             outputs = model(input_ids, masks).last_hidden_state
#             embedding, _ = torch.max(outputs, dim=1)
#             if i:
#                 embeddings = np.vstack((embeddings, embedding.cpu().numpy()))
#             else:
#                 embeddings = embedding.cpu().numpy()
#     return embeddings

In [ ]:
def get_embeddings(texts):
    lsa = make_pipeline(TfidfVectorizer(), TruncatedSVD(n_components=100), Normalizer(copy=False))
    embeddings = lsa.fit_transform(texts)
    return embeddings

In [ ]:
# embeddings = get_embeddings('DeepPavlov/rubert-base-cased-sentence', df['preprocessed_text'].values.astype(str))

In [ ]:
df[text_feature].value_counts().head(20)

In [ ]:
text_feature = 'preproc_description'

In [ ]:
embeddings = get_embeddings(df[text_feature].values)

In [ ]:
with open('embeddings_tfidf_lda100.npy', 'wb') as f:
    np.save(f, embeddings)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN

In [ ]:
 def dbscan_search(data_df, embeddings, eps_grid=[0.2], min_samples_grid=[100], metric='cosine', save_labels=False):
        """
        Clusterize texts using dbscan or optics

        Parameters
        ----------
        eps_grid : list og float, default=[0.2]
            List of maximum distances between two samples of cluster points to try
        min_samples_grid : list of int, default=[100]
            List of neighbor sample count for a point to be considered core
        metric : str, default='cosine'
            The metric to use when calculating distance between instances in a feature array
        column : str, default='PreprocessedText'
            Name of DataFrame column. Contains texts, which are to be clusterized
        """
        fig = plt.figure(figsize=(15, 15))
        coords = PCA(n_components=2).fit_transform(embeddings)
        clusters_df = pd.DataFrame(columns=eps_grid, index=min_samples_grid)
        noise_df = pd.DataFrame(columns=eps_grid, index=min_samples_grid)
        k = 0
        for i, min_samples in enumerate(tqdm(min_samples_grid)):
            for j, eps in enumerate(eps_grid):
                k += 1
                db = DBSCAN(eps=eps, min_samples=min_samples, metric=metric).fit(embeddings)
                noise = coords[db.labels_ == -1]
                clusters = coords[db.labels_ != -1]
                cluster_labels = db.labels_[db.labels_ != -1]
                ax = fig.add_subplot(len(min_samples_grid), len(eps_grid), k)
                clusters_df.iloc[i, j] = len(set(cluster_labels))
                noise_df.iloc[i, j] = len(noise)
                if noise_df.iloc[i, j]:
                    ax.scatter(noise[:, 0], noise[:, 1], c='0.8')
                if clusters_df.iloc[i, j]:
                    scatter = ax.scatter(clusters[:, 0], clusters[:, 1], c=cluster_labels)
                    ax.legend(handles=scatter.legend_elements()[0], labels=list(range(max(db.labels_))))
                ax.set_title(f"eps={eps:.3f} min_samples={min_samples}")
                target_feature = f"dbscan_eps{eps:.2f}_min{min_samples}"
                data_df[target_feature] = db.labels_
                data_df.to_csv(f"{target_feature}.csv")
                centroids = []
                for cl in set(db.labels_):
                    if cl != -1:
                        cl_words, cl_words_counts = np.unique(
                            np.hstack(data_df[data_df[target_feature] == cl]['preprocessed_text'].str.split(' ').values),
                            return_counts=True)
                        cl_words_s = pd.Series(cl_words_counts, index=cl_words).sort_values(ascending=False).head(10)
                        centroid = cl_words_s
                        n_samples = len(data_df[data_df[target_feature] == cl])
                        print(f"\tCluster {cl} \t{n_samples} samples")
                        print(centroid)
                        centroids.append({'centroid': centroid.index.tolist(), 'size': n_samples})
                with open(f'cluster_centers_{target_feature}.json', 'w', encoding='utf-8') as f:
                    json.dump(centroids, f, ensure_ascii=False, indent=4)
                plt.savefig(f'{target_feature}.png')
        print("Noise:\n", noise_df, "\nClusters:\n", clusters_df)

In [ ]:
df.to_csv('preproc_data.csv')

In [ ]:
dbscan_search(df, embeddings, eps_grid=[0.09], min_samples_grid=[10])